## Simple GenAI app using  Lang chain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [5]:
##Data Ingestion from website
from langchain_community.document_loaders import WebBaseLoader

In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this content, it might be helpful to read the following:Data Retention Conceptual DocsUsage Limiting Conceptual DocsnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to its\ncustom nature of billing. If you are on Enterprise plan a

In [11]:
type(docs)

list

In [12]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [13]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this co

In [16]:
len(documents)

18

In [37]:
#converting to vectors using huggung face
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [27]:
query_result=embeddings.embed_query(documents[1].page_content)
query_result

[-0.015126066282391548,
 0.03524816408753395,
 0.01416322123259306,
 0.01377012487500906,
 -0.030852528288960457,
 0.026817448437213898,
 0.045646704733371735,
 0.035583820194005966,
 0.010729732923209667,
 0.08635522425174713,
 -0.051464054733514786,
 -0.0391983687877655,
 -0.04956987872719765,
 -0.00017543381545692682,
 0.002369432244449854,
 -0.01858939230442047,
 0.017104018479585648,
 -0.013414929620921612,
 -0.06709285825490952,
 -0.09327000379562378,
 0.013357121497392654,
 -0.024589786306023598,
 -0.013827988877892494,
 0.05105971544981003,
 0.04815196245908737,
 -0.027520565316081047,
 0.00819490011781454,
 -0.01870027557015419,
 0.07198801636695862,
 0.04423265904188156,
 -0.004853248130530119,
 0.08980999886989594,
 0.028059763833880424,
 0.014430203475058079,
 0.04896371439099312,
 0.06323067098855972,
 -0.05169482156634331,
 -0.058552566915750504,
 -0.024946900084614754,
 0.006894988007843494,
 -0.009161160327494144,
 -0.019744915887713432,
 -0.12321554124355316,
 -0.02102

In [29]:
len(query_result)


384

In [30]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [20]:
vectorstoredb

In [21]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith'

In [40]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [33]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f8080613490>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f808063cdf0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [34]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4o` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}